### Defining my classes

In [1]:
import copy

class Map:
    def __init__(self, n_rows, n_cols):
        self.map = [["." for _ in range(n_cols)] for _ in range(n_rows)]
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.guard_start = None
        self.obstructions = []

    def __repr__(self):
        # if self.map is None:
        #     return "Map is not initialized."
        return "\n".join("".join(row) for row in self.map)
    
    @classmethod
    def load_from_file(cls, filename):
        with open(filename, "r") as f:
            map_loading = []
            for line in f:
                line = line.strip()
                map_loading.append(list(line))
        n_rows = len(map_loading)
        n_cols = len(map_loading[0])
        obj = cls(n_rows, n_cols)
        
        for row, row_list in enumerate(map_loading):
            for col, char in enumerate(row_list):
                if char == "#":
                    obj.place_obstruction(row, col)
                elif char in "<>^v":
                    direction_dict = {"^":0, ">":1, "v":2, "<":3}
                    direction = direction_dict[char]
                    obj.guard_start = (row, col, direction)
                    obj.place_guard(row, col, direction)
        
        return obj

    def is_obstruction(self, row, col):
        if self.map[row][col] == "#":
            return True
        else:
            return False
    
    def place_obstruction(self, row: int, col: int) -> None:
        self.map[row][col] = "#"
        self.obstructions.append((row, col))

    def remove_obstruction(self, row: int, col: int) -> None:
        self.map[row][col] = "."
        self.obstructions.remove((row, col))
    
    def place_guard(self, row, col, direction):
        DIRECTION_SYMBOL = {0: "^", 1: ">", 2: "v", 3: "<"}
        self.map[row][col] = DIRECTION_SYMBOL[direction]

    


class Guard:
    # Directions are North, East, South, West
    DIRECTIONS = [(-1,0), (0,1), (1,0), (0,-1)]
    DIRECTION_SYMBOL = {0: "^", 1: ">", 2: "v", 3: "<"}

    def __init__(self, row, col, direction):
        self.row: int = row
        self.col: int = col
        self.direction: int = direction
        self.path: list[tuple] = []
        self.symbol: str = self.DIRECTION_SYMBOL[direction]
        self.on_map: bool = True

    def move(self, game_map: Map) -> None:
        d_row, d_col = self.DIRECTIONS[self.direction]
        row_new, col_new = self.row + d_row, self.col + d_col
            
        if row_new < 0 or row_new == game_map.n_rows or col_new < 0 or col_new == game_map.n_cols:
            self.path.append((self.row, self.col, self.direction))
            self.on_map = False
            # print(f"Guard left the map at position ({self.row},{self.col})")
        
        elif game_map.is_obstruction(row_new, col_new):
            # print(f"On position ({self.row},{self.col}): Hit an obstruction, turned right")
            self.rotate()

        else:
            self.path.append((self.row, self.col, self.direction))
            game_map.map[self.row][self.col] = "X"
            self.row = row_new
            self.col = col_new
            game_map.place_guard(self.row, self.col, self.direction)

    def rotate(self) -> None:
        self.direction = (self.direction + 1) % 4
        self.symbol = self.DIRECTION_SYMBOL[self.direction]
        # print(f"Guard turned right. New direction is: {self.direction} ({self.symbol})")

    def unique_positions_in_path(self) -> int:
        uniques = []
        for elem in self.path:
            elem = (elem[0], elem[1])
            if elem not in uniques:
                uniques.append(elem)
        return len(uniques)
    
    @property
    def moves_in_loop(self) -> bool:
        if len(self.path) == len(set(self.path)):
            return False
        else:
            return True
    
    def detect_loop_floyd(self, game_map: Map) -> bool:
        # Initialize two pointers at the current position of the Guard
        tortoise = (self.row, self.col, self.direction)
        hare = (self.row, self.col, self.direction)

        # Get one copy of the map for each pointer
        t_map = copy.deepcopy(game_map)
        h_map = copy.deepcopy(game_map)

        # Let the pointers run on their map and check coordinates
        # For this we need to instatiate the two Guards within this method
        t_guard = Guard(*tortoise)
        h_guard = Guard(*hare)

        # Let the Guards move on the map and only exit the loop if either falls 
        # off the map or if they are in the same state
        while True:
            t_guard.move(t_map)
            if not t_guard.on_map:
                return False
            
            for i in range(2):
                h_guard.move(h_map)
                if not h_guard.on_map:
                    return False
            
            if (t_guard.row, t_guard.col, t_guard.direction) == (h_guard.row, h_guard.col, h_guard.direction):
                return True


            

class Game:
    pass

### Solving Part 1

Solution is 5534

In [4]:
my_map = Map.load_from_file("day6_input.txt")
print(my_map)
print(my_map.guard_start)
r, c, d = my_map.guard_start # r, c, d = row, col, direction
my_guard = Guard(r, c, d)
while my_guard.on_map:
#for i in range(10):
    my_guard.move(my_map)
print(my_map)
print(my_guard.unique_positions_in_path())


......##...#...#....#.......#....................##............#.#..#.......#.........................................#...........
..................................#.............#................................#..........##..................#.................
....#......................#................#...........................................#....................................#....
............#..............#...#...#...............#..........#.#....#..........................#....##...........................
....#.............#.....#.....................................................................#...#..........................#....
.................#............#......................#.................#.............#..................#.........................
................#...............#....................#...#..#...#.#.....................................................#.........
..............#.#...............................................................#..

### Towards solving part 2
- generate all permutations of 3 obstructions
- write a function taht checks if the three suffice for catching the guard in a loop
    - order the obstructions by the first elemnt of each tuple
    - first elemnts have to be n, n+1, m, m+1 with m > n+1
    - second elemnts have to be a < b, c = b-1, d < c, d = a-1

In [ ]:
my_test_map = Map.load_from_file("day6_example_input.txt")
# print(my_test_map.obstructions)
# print(my_test_map)
r, c, d = my_test_map.guard_start
my_test_guard = Guard(r, c, d)
while my_test_guard.on_map:
    my_test_guard.move(my_test_map)

lops = my_test_map.get_loop_obstruction_points()

counter = 0
for point in lops:
    if point in my_test_guard.path:
        print(point)
        counter += 1
print(counter)
print(my_test_map)


In [ ]:
for i in range(len(l)):
    print(f"Element {i} of found hits")
    new_map = Map(10,10)
    for elem in l[i]:
        new_map.place_obstruction(elem[0], elem[1])
    print(new_map)
    print("\n\n")

In [ ]:
import itertools

combinations = list(itertools.combinations(my_test_map.obstructions, 3))
print(len(combinations))
print(combinations)

test1 = list(combinations[0])
#test1.sort()
print(test1)

def check_row_criterion(l: list) -> bool:
    l.sort()
    row_distances = []
    for i in range(1, len(l)):
        row_distances.append(l[i][0]-l[i-1][0])
    print(f"Row Distances: {row_distances}")
    if 1 in row_distances:
        return True #l.index(1) # this number is either 0 or 1
    else:
        return False

def check_col_criterion(l: list) -> bool:
    l.sort(key=lambda x: x[1])
    col_distances = []
    for i in range(1, len(l)):
        col_distances.append(l[i][1]-l[i-1][1])
    print(f"Col Distances: {col_distances}")
    if 1 in col_distances:
        return True
    else:
        return False

check_row_criterion(test1)

### Implementing some test cases for testing code snippets and solutions for problem 2

In [ ]:
is_loopable = [(0,4), (1,9), (7,8)] # (6,3), pos C
is_loopable2 = [(3,2), (4,7), (6,1)] # (7,6), pos D
is_loopable3 = [(6,1), (9,6), (8,0)] # (7,7) pos B
is_not_loopable = [(0, 4), (1, 9), (3, 2)]
is_not_loopable2 = [(0, 4), (1, 3), (3, 2)]
is_not_loopable3 = [(1,1), (2,3), (5,0)]

tests = [is_loopable, is_loopable2, is_loopable3, is_not_loopable, is_not_loopable2, is_not_loopable3]



### Alternative Approach to find incomplete rectangles in the obstructions present

Write a new function that determines if the 3 points selected can be truned into a loop trap
- sort tuples based on first element
- calculate row distances
- calculate col distances
- check that at least one row distance is 1
- if the first row distance is 1
    - check that at least one col distance is -1
- else if the second row distance is 1
    - check that at least one col distance is 1

Write another function that determines the missing point to complete the loop trap


In [ ]:
def row_distances(triplet: list[tuple]):
    triplet.sort()
    row_dists = []
    for i in range(1, len(triplet)):
        row_dist = triplet[i][0] - triplet[i-1][0]
        row_dists.append(row_dist)
    return row_dists

def col_distances_AB_missing(triplet: list[tuple]):
    triplet.sort()
    col_dists = []
    for i in range(1, len(triplet)):
        col_dist = triplet[i][1] - triplet[0][1]
        col_dists.append(col_dist)
    return col_dists

def col_distances_CD_missing(triplet: list[tuple]):
    triplet.sort()
    col_dists = []
    for i in range(len(triplet) - 1):
        col_dist = triplet[len(triplet)-1][1] - triplet[i][1]
        col_dists.append(col_dist)
    return col_dists

def check_triplet_loopability(triplet: list[tuple]):
    test_row_distances = row_distances(triplet)
    if test_row_distances[0] == 1:
        state = "CD_missing"
        #print(f"Destected state: {state}")
    elif test_row_distances[1] == 1:
        state = "AB_missing"
        #print(f"Destected state: {state}")
    else:
        #print(f"{test} is not loopable")
        return None
    
    if state == "AB_missing":
        test_col_distances = col_distances_AB_missing(triplet)
        if test_col_distances[0] == -1 and test_col_distances[1] > 1:
            state = "B_missing"
            #print(f"Destected state: {state}")
        elif test_col_distances[1] == -1 and test_col_distances[0] < -1:
            state = "A_missing"
            #print(f"Destected state: {state}")
        else:
            #print(f"{test} is not loopable")
            return None
            
    else:
        test_col_distances = col_distances_CD_missing(triplet)
        if test_col_distances[0] == -1 and test_col_distances[1] < -1:
            state = "D_missing"
            #print(f"Destected state: {state}")
        elif test_col_distances[1] == -1 and test_col_distances[0] > 1:
            state = "C_missing"
            #print(f"Destected state: {state}")
        else:
            #print(f"{test} is not loopable")
            return None
    
    return state
            

def calc_missing_point(state: str, triplet: list[tuple]):
    triplet.sort()
    if state == "A_missing":
        row = triplet[0][0] - 1
        col = triplet[1][1] + 1
    elif state == "B_missing":
        row = triplet[0][0] + 1
        col = triplet[2][1] + 1
    elif state == "C_missing":
        row = triplet[2][0] - 1
        col = triplet[0][1] - 1
    elif state == "D_missing":
        row = triplet[2][0] + 1
        col = triplet[1][1] -1
    else:
        return print("Wrong 'state' provided")
    return (row, col)

In [ ]:
print(f"Found {len(tests)} tests")

for test in tests:
    print(f"\nTesting {test}")
    test_row_distances = row_distances(test)
    if test_row_distances[0] == 1:
        state = "CD_missing"
        print(f"Destected state: {state}")
    elif test_row_distances[1] == 1:
        state = "AB_missing"
        print(f"Destected state: {state}")
    else:
        print(f"{test} is not loopable")
        state = None
        continue
    
    if state == "AB_missing":
        test_col_distances = col_distances_AB_missing(test)
        if test_col_distances[0] == -1 and test_col_distances[1] > 1:
            state = "B_missing"
            print(f"Destected state: {state}")
        elif test_col_distances[1] == -1 and test_col_distances[0] < -1:
            state = "A_missing"
            print(f"Destected state: {state}")
        else:
            print(f"{test} is not loopable")
            state = None
            continue
    else:
        test_col_distances = col_distances_CD_missing(test)
        if test_col_distances[0] == -1 and test_col_distances[1] < -1:
            state = "D_missing"
            print(f"Destected state: {state}")
        elif test_col_distances[1] == -1 and test_col_distances[0] > 1:
            state = "C_missing"
            print(f"Destected state: {state}")
        else:
            print(f"{test} is not loopable")
            state = None
            continue


    

In [ ]:
for triplet in tests:
    state = check_triplet_loopability(triplet)
    if state:
        missing_point = calc_missing_point(state, triplet)
    print(f"Testing: {triplet}")
    print(f"State: {state}")
    if state:
        print(f"The missing point to introduce a loop is {missing_point}\n")
    else:
        print("This triplate is not loopable\n")
    

### Solving Part 2
Solution is 2262

##### Approach to track visited coordinates including directions

No coordinate should be visited twice with the guard facing into the same direction. Once this happens, we will know that the guard is running in a loop.

In [18]:
from tqdm import tqdm

# Load the Map from file
my_test_map = Map.load_from_file("day6_input.txt")

# Get the Guard starting position
r, c, d = my_test_map.guard_start

# Instantiate the Guard based on its starting position
my_test_guard = Guard(r, c, d)

# Let the Guard complete the entire map once to know his path
while my_test_guard.on_map:
    my_test_guard.move(my_test_map)

full_guard_path = my_test_guard.path
print(f"The full path of the Guard on your map is {len(full_guard_path)} steps.")

# Iterate over the entire path, placing an obstructions at each position and checking if this captures the Guard in a loop
loop_coordinates = []

for coordinate in tqdm(full_guard_path[1:]):
    my_test_map.place_obstruction(coordinate[0], coordinate[1])
    new_guard = Guard(r, c, d)
    while new_guard.on_map:
        new_guard.move(my_test_map)
        if new_guard.moves_in_loop:
            loop_coordinates.append((coordinate[0], coordinate[1]))
            break
    my_test_map.remove_obstruction(coordinate[0], coordinate[1])

print(len(set(loop_coordinates)))

The full path of the Guard on your map is 6159 steps.


100%|██████████| 6158/6158 [19:09<00:00,  5.36it/s]

2262


### Using Floyd's tortoise and hare algorithm to detect the loops
This should make calculations a lot faster. Let's see how much time we can save from our previously required 19 minutes.

This implementation finishes after 1:11 min, so we have an almost 17-fold decrease in compute time 🔥

In [3]:
from tqdm import tqdm

# Load the Map from file
my_test_map = Map.load_from_file("day6_input.txt")

# Get the Guard starting position
r, c, d = my_test_map.guard_start

# Instantiate the Guard based on its starting position
my_test_guard = Guard(r, c, d)

# Let the Guard complete the entire map once to know his path
while my_test_guard.on_map:
    my_test_guard.move(my_test_map)

full_guard_path = my_test_guard.path
print(f"The full path of the Guard on your map is {len(full_guard_path)} steps.")

# Get unique positions in the Guard path
unique_path = set()
for coordinate in full_guard_path:
    unique_path.add((coordinate[0], coordinate[1]))

loop_coordinates = []

for coordinate in tqdm(unique_path, desc="Processing Coordinates", unit="coordinates"):
    my_test_map.place_obstruction(*coordinate)

    guard = Guard(r, c, d)

    if guard.detect_loop_floyd(my_test_map):
        loop_coordinates.append(coordinate)

    my_test_map.remove_obstruction(*coordinate)
    
print(f"Found {len(loop_coordinates)} positions that can trap the Guard in a loop.")

The full path of the Guard on your map is 6159 steps.


Processing Coordinates: 100%|██████████| 5534/5534 [01:11<00:00, 77.55coordinate/s]

Found 2262 positions that can trap the Guard in a loop.
